In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import json
import os
from IPython.display import Image, display
os.chdir('..')

In [ ]:
with open('config/results-params.json') as fh:
    results_cfg = json.load(fh)
dims = results_cfg['dims']
outdir = results_cfg['outdir']

### Data

In [ ]:
polarities = pd.read_csv(os.path.join(results_cfg['user_data_path'], 'polarities.csv')).dropna().drop(columns=['Unnamed: 0'])
polarities.head()

In [ ]:
polarities['user/name'].unique()

In [ ]:
polarities['flagged'].value_counts()

### Exploratory Analysis

In [ ]:
data = polarities[['flagged'] + results_cfg['dims']]
data.describe()

In [ ]:
data.head()

In [ ]:
for dim in results_cfg['dims']:
    display(Image(filename=os.path.join(results_cfg['outdir'], 'kde_plot_{}.png'.format(dim))))

### Examining Correlation  
This section looks at whether there is a quantifiable and/or visual correlation between each of the three dimensions. Each of the below plots is made up of points, each one representing a user. The color of the point indicates whether that user was one who retweeted a flagged tweet or an unflagged tweet.

In [ ]:
for i in range(len(dims)):
    for j in range(i+1, len(dims)):
        display(Image(filename=os.path.join(results_cfg['outdir'], 'correlation_{}_{}.png'.format(dims[i], dims[j]))))

Correlations

In [ ]:
polarities[dims].corr()

### Test 1
I will perform a permutation test for each dimension. That is, for one dimension I will calculate the difference between the sample means for all polarity scores for the flagged tweets group and then for the unflagged tweets groups. I will define the test statistic as the difference between those two means. I will then shuffle the flagged/unflagged labels and then recalculate the statistic. I will do this 100 times then calculate the p value.  
  
**Permutation Test:**  
  
Test Statistic: Define the test statistic as the difference between the sample means for the flagged and unflagged group for one dimension  
  
Null Hypothesis: There is no difference in the polarity scores between the flagged and unflagged groups  
  
Alternative Hypothesis: The flagged users have a higher polarity score in a respective dimension than the unflagged users

In [ ]:
for dim in dims:
    with open(os.path.join(outdir, 'perm_test_{}.json'.format(dim))) as fh:
        cfg = json.load(fh)
        print(cfg)

**Evaluating:**  
The break down of the tests for each dimension is reported above. Notice that for the credibility and the moderacy dimensions, there is a more significant difference between the two groups than there is for the poltical dimension. Hopefully with more data this becomes more apparent. 

### Test 2
We will compare the data for each dimension using a t-test. Under this test we assume the data is normally distributed with the same variance. We will perform a two-sided test between the flagged and unflagged group for each dimension.  
  
**Two-sided t-test**  

In [ ]:
for dim in dims:
    with open(os.path.join(outdir, 'two_sided_ttest_{}.json'.format(dim))) as fh:
        cfg = json.load(fh)
        print(cfg)

**Outcome** 
We can see a very similar outcome as the permutation test, as we would expect. Next we will test the one-sided t-test for both groups

### Test 3
We will test whether flagged group has statistically significantly higher or lower polarity scores than the unflagged group for both dimensions. We will use a one-sided t-test.  
**One-sided t-test**

In [ ]:
for dim in dims:
    with open(os.path.join(outdir, 'one_sided_ttest_{}.json'.format(dim))) as fh:
        cfg = json.load(fh)
        print(cfg)

**Outcome**  
First, let alpha = 0.1. Next, we can derive the outcome of the one-sided as the flagged polarities are significantly greater than the unflagged polarities if the p-value < alpha and the test statistic is > 0. Likewise if the test statistic is < 0 then it means the flagged polarities are significantly lesser than the unflagged polarities. Notice here that while there is no statistically significant outcome for the moderacy and credibility dimensions, we can see that the political polarity of users who retweeted a flagged tweet is statistically significantly greater than those of the users who retweeted an unflagged tweet. This is what we would expect since the tweets being flagged are coming from a user who is generally aligned with right wing views so the users that are interacting with a flagged tweet tend to have a higher polarity tending towards the right.  